# LAB 01: LAYOFFS 2023

## Mô tả đồ án:
- Từ nguồn dữ liệu lấy được từ Kaggle về vấn đề sa thải nhân viên ở các công ty trên toàn thế giới, đồ án này nhằm mục đích phân tích dữ liệu đã có để có được cái nhìn khách quan hơn về vấn đề này.

---

### Import các thư viện

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

---

### <span style="color:yellow"> A. Thu thập dữ liệu</span>


#### Ngữ cảnh thực hiện tìm kiếm dữ liệu

Sau dịch covid 19, các công ty công nghệ trên toàn cầu đang đối mặt với sự suy thoái kinh tế. Sự chậm lại của chi tiêu từ người tiêu dùng, lãi suất cao hơn từ các ngân hàng trung ương và sức mạnh của đồng đô la ngoại tệ đang gợi ý về khả năng xảy ra suy thoái kinh tế và các công ty công nghệ đã bắt đầu **cắt giảm nhân sự**.

#### Chủ đề

Phân tích tình trạng hỗn loạn công nghệ gần đây thông qua số liệu về việc sa thải nhân sự, từ đó rút ra được những thông tin hữu ích.

#### Nguồn
Dữ liệu được lấy trên Kaggle dưới file csv, đường link dẫn đến  [Ở đây](https://www.kaggle.com/datasets/swaptr/layoffs-2022?resource=download)

#### Người ta có cho phép sử dụng dữ liệu như thế này hay không?

Thông tin License của dữ liệu: Open Database.

Theo thông tin License của dữ liệu, người dùng được phép tự do chia sẻ, sửa đổi và sử dụng dữ liệu trong khi duy trì sự tự do tương tự cho người dùng khác.

#### Người ta đã thu thập dữ liệu này như thế nào? Phương pháp thực hiện là gì?

Dữ liệu được thu thập bằng cách theo dõi số liệu được báo cáo trên các nền tảng:
- Bloombreg
- San Francisco Business Times
- TechCrunch
- The New York Times

Phương pháp thu thập dữ liệu: Dữ liệu được tải xuống từ Layoffs.fyi Tracker và được biên soạn thành 1 file CSV.

#### Đọc dữ liệu

In [2]:
df=pd.read_csv('./datasets/layoffs.csv')
df.head()

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised
0,Atlassian,Sydney,Other,500.0,0.05,2023-03-06,Post-IPO,Australia,210.0
1,SiriusXM,New York City,Media,475.0,0.08,2023-03-06,Post-IPO,United States,525.0
2,Alerzo,Ibadan,Retail,400.0,NaN,2023-03-06,Series B,Nigeria,16.0
3,UpGrad,Mumbai,Education,120.0,NaN,2023-03-06,Unknown,India,631.0
4,Loft,Sao Paulo,Real Estate,340.0,0.15,2023-03-03,Unknown,Brazil,788.0


---

### <span style="color:yellow"> B. Khám phá dữ liệu</span>


#### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

In [3]:
# df['stage']

- Mỗi dòng chứa dữ liệu về một công ty và các thông tin liên quan đến công ty đó như: tên công ty, nơi đặt trụ sở chính, loại ngành nghề, số nhân viên bị sa thải, tỷ lệ sa thải, ngày công bố, giai đoạn của công ty, quốc gia công ty hoạt động, số vốn huy động bởi công ty.

- ví dụ: dòng 1: doanh nghiệp SiriusXM có trụ sở tại New York City và hoạt động trong ngành truyền thông, đã sa thải 475 nhân viên, chiếm 8% tổng số nhân viên của họ tính tới ngày 6 tháng 3 năm 2023. Công ty cũng đang ở giai đoạn Post-IPO và có trụ sở tại Hoa Kỳ. Họ đã huy động được số vốn 525 triệu đô la.

- Không có tình trạng các dòng có ý nghĩa khác nhau.

#### Mỗi cột có ý nghĩa gì?

In [4]:

# print("total columns: ",len(df.columns)) 

# for column in range(len(df.columns)) :
#     print("column ", column," ", df.columns[column])
#     print("size: ", len(df.loc[:, df.columns[column]]))
#     print("number of values: ", len(df.loc[:, df.columns[column]].unique()), "\n")
#     print("head: ", ' ,'.join(df.loc[:, df.columns[column]].unique()) )

column=2
print("column ", column,": ", df.columns[column])
print("size: ", len(df.loc[:, df.columns[column]]))
print("number of values: ", len(df.loc[:, df.columns[column]].unique()))
# print(df.loc[:, df.columns[column]].unique())
arr= df.loc[:, df.columns[column]].unique()

nanIndex=-1
for index in range(len(arr)):
    if isinstance(arr[index], float) and np.isnan(arr[index]):
        nanIndex=index
        break
if nanIndex != -1:
    arr=np.delete(arr,nanIndex)

print("number of values (without nan): ", len(arr))

# print for string type
print("data (without nan) : ", ', '.join(arr))

# print for number type
# print( ", ".join(map(str, arr)))

    

column  2 :  industry
size:  2357
number of values:  30
number of values (without nan):  29
data (without nan) :  Other, Media, Retail, Education, Real Estate, Transportation, Marketing, Travel, Healthcare, Security, Food, Fitness, Consumer, Logistics, HR, Support, Crypto, Finance, Data, Sales, Infrastructure, Hardware, Product, Construction, Legal, Energy, Manufacturing, Recruiting, Aerospace


## Thông tin của mỗi cột:
- Company: Tên của công ty
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 1893
    - Một số Company có trong dữ liệu: Atlassian, SiriusXM, Alerzo, UpGrad, Loft, Embark Trucks,...
- Location: Nơi đặt trụ sở chính của công ty
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 191
    - Một số Location có trong dữ liệu: Sydney, New York City, Ibadan, Mumbai, Sao Paulo,...

- Industry: Loại ngành mà công ty hoạt động
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 30
    - Một số Industry có trong dữ liệu: Other, Media, Retail, Education, Real Estate, Transportation,...

- Total_laid_off: Tổng số nhân viên bị sa thải bởi công ty
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 286
    - Một số Total_laid_off có trong dữ liệu: 475.0, 400.0, 120.0, 340.0, 230.0, 100.0, 63.0, 30.0...

- Percentage_laid_off: Tỷ lệ nhân viên bị sa thải bởi công ty
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 76
    - Một số Percentage_laid_off có trong dữ liệu: 0.05, 0.08, 0.15, 0.7, 0.03, 0.11, 0.1, 1.0...

- Date: Ngày công bố thông tin về việc sa thải
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 191
    - Một số Date có trong dữ liệu: 2023-03-06, 2023-03-03, 2023-03-02, 2023-03-01, 2023-02-28,...

- Stage: Giai đoạn của công ty
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 17
    - Một số Stage có trong dữ liệu: Post-IPO, Series B, Unknown, Acquired, Series F, Series E...

- Country: Quốc gia nơi công ty đặt trụ sở
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 59
    - Một số Country có trong dữ liệu:  Australia, United States, Nigeria, India, Brazil,...

- Funds_raised: Tổng số vốn do công ty huy động (tính bằng triệu đô la)
    - Kích thước: 2357
    - Số lượng dữ liệu không trùng lắp: 639
    - Một số Funds_raised có trong dữ liệu: 525.0, 16.0, 631.0, 788.0, 317.0, 59.0, 152.0...

#### Mỗi cột hiện đang có kiểu dữ liệu gì?

In [5]:
df.dtypes

company                 object
location                object
industry                object
total_laid_off         float64
percentage_laid_off    float64
date                    object
stage                   object
country                 object
funds_raised           float64
dtype: object

In [6]:
object_columns = df.select_dtypes(include=['object']).columns
for col in object_columns:
    print(col,":",set(df[col].apply(type)))

company : {<class 'str'>}
location : {<class 'str'>}
industry : {<class 'str'>, <class 'float'>}
date : {<class 'str'>, <class 'float'>}
stage : {<class 'str'>, <class 'float'>}
country : {<class 'str'>}


Chúng ta nhận thấy rằng ba cột `industry`, `date` và `stage` có dạng dữ liệu không đồng nhất, tuy nhiên đây chỉ là do sự thiếu hụt dữ liệu trong các dòng, được biểu thị bằng giá trị `NaN` (dạng số thực).

#### Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?

Các cột sau đang có kiểu dữ liệu là `object`, do đó cần được chuyển đổi sang kiểu chuỗi (`string`):

* `company`
* `location`
* `industry`
* `stage`
* `country`

Trong khi đó, cột `date` đang có kiểu dữ liệu là `object`, cần được chuyển đổi sang kiểu ngày tháng (`datetime`).

#### Có cần phải tiền xử lý dữ liệu hay không và nếu có thì nhóm sinh viên cần phải xử lý như thế nào?

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2357 entries, 0 to 2356
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   company              2357 non-null   object 
 1   location             2357 non-null   object 
 2   industry             2356 non-null   object 
 3   total_laid_off       1618 non-null   float64
 4   percentage_laid_off  1573 non-null   float64
 5   date                 2356 non-null   object 
 6   stage                2351 non-null   object 
 7   country              2357 non-null   object 
 8   funds_raised         2148 non-null   float64
dtypes: float64(3), object(6)
memory usage: 165.9+ KB


Quan sát thấy có số lượng lớn dữ liệu bị thiếu và có những cột phải xử lý kiểu dữ liệu do vậy ta phải tiền xử lý dữ liệu.

### Tiền xử lý dữ liệu

In [8]:
# Kiểm tra missing data
have_Nan=df.columns[df.isna().any()]

# Đếm số missing data của các cột có missing value
print("Missing data:")
print(df[have_Nan].isnull().sum())

# Phần trăm dữ liệu bị thiểu của những cột này
print("Percentage missing data:")
print(df[have_Nan].isnull().sum() * 100 / df.shape[0])

# Kiểm tra có dữ liệu bị duplicate không
print("Number of rows duplicated: "+str(df[df.duplicated()].shape[0]))


Missing data:
industry                 1
total_laid_off         739
percentage_laid_off    784
date                     1
stage                    6
funds_raised           209
dtype: int64
Percentage missing data:
industry                0.042427
total_laid_off         31.353415
percentage_laid_off    33.262622
date                    0.042427
stage                   0.254561
funds_raised            8.867204
dtype: float64
Number of rows duplicated: 1


Ta thấy có một số lượng khá lớn dữ liệu bị thiếu ở cột `total_laid_off`,`percentage_laid_off` và `funds_raised` . Vì có những cột dữ liệu bị thiếu lên đến hơn 30% nên ta không thể chọn phương án bỏ toàn bộ những dòng nào bị thiếu dữ liệu. Bên cạnh đó có tồn tại 1 dòng bị trùng ta cũng cần phải xử lý. 

- Đầu tiên ta loại bỏ những dòng có dữ liệu bị thiếu ở cột `industry`,`date` và `stage` vì số lượng bị thiếu không đáng kể.
- Xóa dòng bị trùng dữ liệu.
- Loại bỏ những dòng mà bị thiếu cả hai trường quan trọng là `total_laid_off` và `percentage_laid_off` vì những dòng này không có ý nghĩa để ta phân tích.
- Tiếp theo tùy theo hướng phân tích ta sẽ chọn những cột dữ liệu phù hợp. tạm thời ta sẽ fill bằng 0.

In [9]:
# Loại bỏ những dòng có dữ liệu bị thiếu ở cột `industry`,`date` và `stage`
df = df[~df['industry'].isna()]
df = df[~df['date'].isna()]
df = df[~df['stage'].isna()]

# Xóa dòng bị trùng dữ liệu.
df = df[~df.duplicated()]

# Loại bỏ những dòng mà bị thiểu cả hai trường quan trọng là `total_laid_off` và `percentage_laid_off`
df = df[~(df['total_laid_off'].isna()&df['percentage_laid_off'].isna())]

# 
df = df.fillna(0)
print("After fill null:")
print(df.isna().sum())

After fill null:
company                0
location               0
industry               0
total_laid_off         0
percentage_laid_off    0
date                   0
stage                  0
country                0
funds_raised           0
dtype: int64


Sau khi đã tạm thời xử lý các dữ liệu bị thiếu ta bắt đầu xử lý kiễu dữ liệu của các cột.

- Chuyển data type của cột `date` sang dạng `datetime`.
- Chuyển data type của các cột đang là dạng `object ('str')` sang dạng `category`

In [10]:
# Chuyển data type của cột `date` sang dạng `datetime`.
df['date']=pd.to_datetime(df['date'])
# Chuyển data type của các cột đang là dạng `object` sang dạng `category`
object_columns = df.select_dtypes(include=['object']).columns
for col in object_columns:
    df[col]=df[col].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1988 entries, 0 to 2355
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   company              1988 non-null   category      
 1   location             1988 non-null   category      
 2   industry             1988 non-null   category      
 3   total_laid_off       1988 non-null   float64       
 4   percentage_laid_off  1988 non-null   float64       
 5   date                 1988 non-null   datetime64[ns]
 6   stage                1988 non-null   category      
 7   country              1988 non-null   category      
 8   funds_raised         1988 non-null   float64       
dtypes: category(5), datetime64[ns](1), float64(3)
memory usage: 178.3 KB


Đã xữ lý xong kiểu dữ liệu của các cột. Giờ ta quan sát đến giá trị dữ liệu bên trong các cột.

In [11]:
# Sắp xếp df theo tên company để quan sát cột `company`
print(df.sort_values('company')['company'].unique())
a=df['company'].unique()
b=df['company'].str.upper().unique()
print(a.shape[0])
print(b.shape[0])

[' Included Health', '#Paid', '&Open', '100 Thieves', '10X Genomics', ..., 'iFood', 'iPrice Group', 'iRobot', 'nCino', 'uShip']
Length: 1629
Categories (1629, object): [' Included Health', '#Paid', '&Open', '100 Thieves', ..., 'iPrice Group', 'iRobot', 'nCino', 'uShip']
1629
1624


Số lượng tên công ty:

-   Data gốc: 1629
-   Data sau khi đưa tất cả về uppercase: 1624

Ta thấy có sự khác nhau về số lượng tên công ty khi ta giữ nguyên data gốc và sau khi đưa tất cả về uppercase -> Có những tên của cùng một công ty nhưng chưa được chuẩn hóa cùng 1 quy tắc.
-> Đưa tất cả về 1 loại (uppercase) và ta cũng phải xóa các khoảng trắng thừa trong tên công ty phòng trường hợp có tên công ty bị thừa khoảng trắng.

In [12]:
# Xử lý dữ liệu cột `company`
df['company']=df['company'].str.upper()
df['company']=df['company'].str.split().str.join(" ")
df['company'].unique().shape[0]

1621

Xử lý tương tự cho cột `location`, `industry`, `stage` và `country`.

In [13]:
# Xử lý dữ liệu cột `location`
df['location']=df['location'].str.upper()
df['location']=df['location'].str.split().str.join(" ")
df['location'].unique().shape[0]

166

In [14]:
# Xử lý dữ liệu cột `industry`
df['industry']=df['industry'].str.upper()
df['industry']=df['industry'].str.split().str.join(" ")
df['industry'].unique().shape[0]

29

In [15]:
# Xử lý dữ liệu cột `stage`
df['stage']=df['stage'].str.upper()
df['stage']=df['stage'].str.split().str.join(" ")
df['stage'].unique().shape[0]

16

In [16]:
# Xử lý dữ liệu cột `country`
df['country']=df['country'].str.upper()
df['country']=df['country'].str.split().str.join(" ")
df['country'].unique().shape[0]

51

Tiếp theo ta kiểm tra những cột có kiễu dữ liệu là số để xem có xuất hiện dữ liệu không hợp lệ không.
- Có dòng nào chứa số âm không?
- `percentage_laid_off` có giá trị nào lớn hơn 1 không?

In [17]:
# Kiểu tra giá trị âm
print("Negative total_laid_off: "+str(sum(df.total_laid_off<0)))
print("Negative percentage_laid_off: "+str(sum(df.percentage_laid_off<0)))
print("Negative funds_raised: "+str(sum(df.funds_raised<0)))

# Kiểm tra giá trị `percentage_laid_off` không hợp lệ
print("percentage_laid_off greater than 1: "+str(sum(df.percentage_laid_off>1)))

Negative total_laid_off: 0
Negative percentage_laid_off: 0
Negative funds_raised: 0
percentage_laid_off greater than 1: 0


Sau khi đã xử lý xong tất cả ta sắp xếp và reset index của data.

In [18]:
df=df.sort_values('company')
df=df.reset_index(drop=True)
df.head()

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised
0,#PAID,TORONTO,MARKETING,19.0,0.17,2023-01-27,SERIES B,CANADA,21.0
1,&OPEN,DUBLIN,MARKETING,9.0,0.09,2022-11-17,SERIES A,IRELAND,35.0
2,100 THIEVES,LOS ANGELES,CONSUMER,12.0,0.00,2022-07-13,SERIES C,UNITED STATES,120.0
3,10X GENOMICS,SF BAY AREA,HEALTHCARE,100.0,0.08,2022-08-04,POST-IPO,UNITED STATES,242.0
4,1STDIBS,NEW YORK CITY,RETAIL,70.0,0.17,2020-04-02,SERIES D,UNITED STATES,253.0


Tới đây ta đã tiền xử lý dữ liệu xong tạm ổn.

In [19]:

# Khúc này là thêm 1 cột `total_employees` mục đích là lấy ra top company, lưu tạm để qua phần phân tích dùng
df_with_total_employees=df.sort_values(by='total_laid_off',ascending=False)
df_with_total_employees=df_with_total_employees.reset_index(drop=True)
df_with_total_employees=df_with_total_employees[df_with_total_employees['percentage_laid_off']>0]
df_with_total_employees['total_employees']=round(df_with_total_employees['total_laid_off']/df_with_total_employees['percentage_laid_off'])
df_with_total_employees=df_with_total_employees[df_with_total_employees['total_employees']>0]
df_with_total_employees=df_with_total_employees.sort_values(by='total_employees',ascending=False)
df_with_total_employees=df_with_total_employees.reset_index(drop=True)
df_with_total_employees


,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised,total_employees
0,AMAZON,SEATTLE,RETAIL,8000.0,0.02,2023-01-04,POST-IPO,UNITED STATES,108.0,400000.0
1,AMAZON,SEATTLE,RETAIL,10000.0,0.03,2022-11-16,POST-IPO,UNITED STATES,108.0,333333.0
2,GOOGLE,SF BAY AREA,CONSUMER,12000.0,0.06,2023-01-20,POST-IPO,UNITED STATES,26.0,200000.0
3,MICROSOFT,SEATTLE,OTHER,10000.0,0.05,2023-01-18,POST-IPO,UNITED STATES,1.0,200000.0
4,IBM,NEW YORK CITY,HARDWARE,3900.0,0.02,2023-01-25,POST-IPO,UNITED STATES,0.0,195000.0
...,...,...,...,...,...,...,...,...,...,...
1184,DOTSCIENCE,LONDON,PRODUCT,10.0,1.00,2020-05-19,UNKNOWN,UNITED KINGDOM,0.0,10.0
1185,PANDA SQUAD,SF BAY AREA,CONSUMER,6.0,0.75,2020-03-13,SEED,UNITED STATES,1.0,8.0
1186,ATSU,SEATTLE,INFRASTRUCTURE,6.0,1.00,2020-04-10,UNKNOWN,UNITED STATES,1.0,6.0
1187,DARK,SF BAY AREA,PRODUCT,6.0,1.00,2020-06-23,SEED,UNITED STATES,3.0,6.0


In [20]:
df2=df_with_total_employees.groupby('company')['total_employees'].max()
df2.sort_values(ascending=False).head(10)

company
AMAZON        400000.0
GOOGLE        200000.0
MICROSOFT     200000.0
IBM           195000.0
DELL          133000.0
ERICSSON      106250.0
SAP           100000.0
SALESFORCE    100000.0
META           84615.0
CISCO          82000.0
Name: total_employees, dtype: float64